In [1]:
# Подключаем необходимые для работы библиотеки
import numpy as np
import pandas as pd
import os
import dask.dataframe as dd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Загружаем в DataFrame данные из файла csv с закупками
df_original = pd.read_csv('/content/drive/MyDrive/fz-100.csv', encoding='utf-8', on_bad_lines="skip", delimiter=',', names=['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Subject','Budget','B2','N3','Date1','Date2','Date3','N4','RejectReason','RR2','Sum','S2','Additional','Cat1','Cat2','OKPD2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'])
# df_original = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fz.zip', encoding='utf-8', on_bad_lines="skip", compression='zip', delimiter=',', names=['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Subject','Budget','B2','N3','Date1','Date2','Date3','N4','RejectReason','RR2','Sum','S2','Additional','Cat1','Cat2','OKPD2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'])
# df_original = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/fz.zip', encoding='utf-8', on_bad_lines="skip", blocksize=None, compression='zip', delimiter=',', names=['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Subject','Budget','B2','N3','Date1','Date2','Date3','N4','RejectReason','RR2','Sum','S2','Additional','Cat1','Cat2','OKPD2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'])

<ipython-input-2-ac7209c4bc77>:2: DtypeWarning: Columns (28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv('/content/drive/MyDrive/fz-100.csv', encoding='utf-8', on_bad_lines="skip", delimiter=',', names=['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Subject','Budget','B2','N3','Date1','Date2','Date3','N4','RejectReason','RR2','Sum','S2','Additional','Cat1','Cat2','OKPD2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'])


In [ ]:
df_out = pd.DataFrame()
with pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fz.zip', encoding='utf-8', on_bad_lines="skip", compression='zip', delimiter=',', chunksize=1000000, names=['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Subject','Budget','B2','N3','Date1','Date2','Date3','N4','RejectReason','RR2','Sum','S2','Additional','Cat1','Cat2','OKPD2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14']) as reader:
 for chunk in reader:
  df_work = chunk
  df_work = df_work.drop(['Number','N1','N2','INN','Contractor','BIK','Customer','Region','R2','Performed','Budget','B2','N3','Date3','N4','RejectReason','RR2','S2','Additional','Cat1','Cat2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'], axis=1)
  df_work = df_work.dropna(subset=['OKPD2'])
  df_work = df_work.loc[df_work['Subject'] != '<НЕ ОПРЕДЕЛЕНО>']
  df_41_OKPD = df_work[df_work['OKPD2'].str.contains('^41.')]
  df_42_OKPD = df_work[df_work['OKPD2'].str.contains('^42.')]
  df_43_OKPD = df_work[df_work['OKPD2'].str.contains('^43.')]
  df_71_OKPD = df_work[df_work['OKPD2'].str.contains('^71.1')]
  df_out = pd.concat([df_out, df_41_OKPD], ignore_index=True)
  df_out = pd.concat([df_out, df_42_OKPD], ignore_index=True)
  df_out = pd.concat([df_out, df_43_OKPD], ignore_index=True)
  df_out = pd.concat([df_out, df_71_OKPD], ignore_index=True)
  print(len(df_out))

In [14]:
print(len(df_out))

228066


In [4]:
df_work = df_original

In [34]:
df_out.sample(20)

,Subject,Date1,Date2,Sum,OKPD2
161339,Контракт заключен,2022-07-19,2022-12-31,32834.00,43.2
163719,Контракт исполнен,2022-05-31,2022-12-31,709652.40,43.2
121327,Исполнение контракта прекращено,2021-08-03,2021-12-31,908494.94,42.1
35171,Контракт заключен,2022-05-04,2022-12-31,8923279.58,41.2
45058,Контракт заключен,2022-07-18,2022-11-10,3479545.20,42.2
163901,Контракт исполнен,2022-07-11,2022-11-01,300000.00,43.2
202019,Контракт заключен,2022-03-28,2022-12-31,899431.00,42.1
143081,Контракт заключен,2022-08-19,2022-12-31,1250598.00,42.9
139312,Контракт заключен,2022-08-19,2022-12-31,7072573.00,42.1
122733,Контракт исполнен,2022-04-04,2022-12-31,1686000.00,42.1


In [16]:
df_out.shape

(228066, 5)

In [17]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228066 entries, 0 to 228065
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Subject  228066 non-null  object
 1   Date1    228064 non-null  object
 2   Date2    228066 non-null  object
 3   Sum      228066 non-null  object
 4   OKPD2    228066 non-null  object
dtypes: object(5)
memory usage: 8.7+ MB


In [8]:
df_work = df_work.drop(['Number','N1','N2','INN','BIK','Region','R2','Performed','Budget','B2','N3','N4','RejectReason','RR2','S2','Additional','Cat1','Cat2','O2','N5','N6','N7','N8','N9','N10','N11','N12','N13','N14'], axis=1)

In [9]:
df_work = df_work.dropna(subset=['OKPD2'])

In [10]:
df_work = df_work.loc[df_work['Subject'] != '<НЕ ОПРЕДЕЛЕНО>']

In [11]:
df_work = df_work.reset_index()

In [12]:
del df_work['index']

In [80]:
df_work.loc[df_work['OKPD2'] == '41']

,Contractor,Customer,Subject,Date1,Date2,Date3,Sum,OKPD2
npartitions=1,,,,,,,,
,string,int64,string,string,string,string,string,string
,...,...,...,...,...,...,...,...
